In [ ]:
import sys; sys.path.extend(["../src", ".."])
import sensai
import pandas as pd
import numpy as np
from typing import *
import config

cfg = config.get_config()
sensai.util.logging.configureLogging()

# Neural Networks

Neural networks being a very powerful class of models, especially in cases where the learning of representations from low-level information (such as pixels, audio samples or text) is key, sensAI provides many useful abstractions for dealing with this class of models, facilitating data handling, learning and evaluation.

sensAI mainly provides abstractions for PyTorch, but there is also rudimentary support for TensorFlow.

## Image Classification

As an example use case, let us solve the classification problem of classifying digits in pixel images from the MNIST dataset. Images are greyscale (no colour information) and 28x28 pixels in size.

In [ ]:
mnistDF = pd.read_csv(cfg.datafile_path("mnist_train.csv.zip"))

The data frame contains one column for every pixel, each pixel being represented by an 8-bit integer (0 to 255).

In [ ]:
mnistDF.head(5)

Let's create the I/O data for our experiments.

In [ ]:
mnistIoData = sensai.InputOutputData.fromDataFrame(mnistDF, "label")

Now that we have the image data separated from the labels, let's write a function to restore the 2D image arrays and take a look at some of the images.

In [ ]:
import matplotlib.pyplot as plt

def reshape2DImage(series):
    return series.values.reshape(28, 28)

fig, axs = plt.subplots(nrows=1, ncols=5, figsize=(10, 5))
for i in range(5):
    axs[i].imshow(reshape2DImage(mnistIoData.inputs.iloc[i]), cmap="binary")

### Applying Predefined Models



We create an evaluator in order to test the performance of our models, randomly splitting the data.

In [ ]:
evaluatorParams = sensai.evaluation.VectorClassificationModelEvaluatorParams(fractionalSplitTestFraction=0.2)
evalUtil = sensai.evaluation.ClassificationEvaluationUtil(mnistIoData, evaluatorParams=evaluatorParams)

One pre-defined model we could try is a simple multi-layer perceptron. A PyTorch-based implementation is provided via class `MultiLayerPerceptronVectorClassificationModel`. This implementation supports CUDA-accelerated computations (on Nvidia GPUs), yet we shall stick to CPU-based computation (cuda=False) in this tutorial.

In [ ]:
import sensai.torch

nnOptimiserParams = sensai.torch.NNOptimiserParams(earlyStoppingEpochs=10, batchSize=54)
torchMLPModel = sensai.torch.models.MultiLayerPerceptronVectorClassificationModel(hiddenDims=(50, 20), cuda=False,
        normalisationMode=sensai.torch.NormalisationMode.MAX_ALL, nnOptimiserParams=nnOptimiserParams, pDropout=0.0).withName("MLP")


Neural networks work best on **normalised inputs**, so we have opted to apply basic normalisation by specifying a normalisation mode which will transforms inputs by dividing by the maximum value found across all columns in the training data. For more elaborate normalisation options, we could have used a data frame transformer (DFT), particularly `DFTNormalisation` or `DFTSkLearnTransformer`.

sensAI's default **neural network training algorithm** is based on early stopping, which involves checking, in regular intervals, the performance of the model on a validation set (which is split from the training set) and ultimately selecting the model that performed best on the validation set. You have full control over the loss evaluation method used to select the best model (by passing a respective `NNLossEvaluator` instance to NNOptimiserParams) as well as the method that is used to split the training set into the actual training set and the validation set (by adding a `DataFrameSplitter` to the model or using a custom `TorchDataSetProvider`).

Given the vectorised nature of our MNIST dataset, we can apply any type of model which can accept the numeric inputs. Let's compare the neural network we defined above against another pre-defined model, which is based on a scikit-learn implementation and uses decision trees rather than neural networks.

In [ ]:
randomForestModel = sensai.sklearn.classification.SkLearnRandomForestVectorClassificationModel(min_samples_leaf=1, n_estimators=10) \
    .withName("RandomForest")

evalUtil.compareModels([randomForestModel, torchMLPModel])

Both models perform reasonably well.

### Creating a Custom CNN Model

Given that this is an image recognition problem, it can be sensible to apply convolutional neural networks (CNNs), which can analyse smaller patches of the image in order to generate more high-level features from them.

To define a custom neural network model that uses PyTorch, we need to implement a new model class. For classification and regression, sensAI provides the base classes `TorchVectorClassificationModel` and `TorchVectorRegressionModel` respectively. Ultimately, these classes will wrap an instance of `torch.nn.Module`, the base class for neural networks in PyTorch.

In the following, we shall define a model which uses multiple convolutions, a max-pooling layer and a multi-layer perceptron at the end in order to produce the classification.

In [ ]:
import torch

class CnnModel(sensai.torch.TorchVectorClassificationModel):
    def __init__(self, cuda: bool, kernelSize: int, numConv: int, poolingKernelSize: int, mlpHiddenDims: Sequence[int], 
            nnOptimiserParams: sensai.torch.NNOptimiserParams, pDropout=0.0):
        self.cuda = cuda
        self.outputActivationFn = sensai.torch.ActivationFunction.LOG_SOFTMAX
        self.kernelSize = kernelSize
        self.numConv = numConv
        self.poolingKernelSize = poolingKernelSize
        self.mlpHiddenDims = mlpHiddenDims
        self.pDropout = pDropout
        super().__init__(sensai.torch.ClassificationOutputMode.forActivationFn(self.outputActivationFn),
            modelClass=self.VectorTorchModel, modelArgs=[self], nnOptimiserParams=nnOptimiserParams)
        self.withInputTensoriser(self.InputTensoriser())

    class InputTensoriser(sensai.torch.RuleBasedTensoriser):
        def _tensorise(self, df: pd.DataFrame) -> Union[torch.Tensor, List[torch.Tensor]]:
            images = [reshape2DImage(row) for _, row in df.iterrows()]
            return torch.tensor(np.stack(images)).float() / 255

    class VectorTorchModel(sensai.torch.VectorTorchModel):
        def __init__(self, vecModel: "CnnModel"):
            super().__init__(vecModel.cuda)
            self._vecModel = vecModel

        def createTorchModuleForDims(self, inputDim: int, outputDim: int) -> torch.nn.Module:
            return self.Module(int(np.sqrt(inputDim)), outputDim, self._vecModel)

        class Module(torch.nn.Module):
            def __init__(self, imageDim, outputDim, vecModel: "CnnModel"):
                super().__init__()
                k = vecModel.kernelSize
                p = vecModel.poolingKernelSize
                self.cnn = torch.nn.Conv2d(1, vecModel.numConv, (k, k))
                self.pool = torch.nn.MaxPool2d((p, p))
                self.dropout = torch.nn.Dropout(p=vecModel.pDropout)
                reducedDim = (imageDim-k+1)/p
                if int(reducedDim) != reducedDim:
                    raise ValueError(f"Pooling kernel size {p} is not a divisor of post-convolution dimension {imageDim-k+1}")
                self.mlp = sensai.torch.models.MultiLayerPerceptron(vecModel.numConv * int(reducedDim)**2, outputDim, vecModel.mlpHiddenDims,
                    outputActivationFn=vecModel.outputActivationFn.getTorchFunction(),
                    hidActivationFn=sensai.torch.ActivationFunction.RELU.getTorchFunction(),
                    pDropout=vecModel.pDropout)

            def forward(self, x):
                x = self.cnn(x.unsqueeze(1))
                x = self.pool(x)
                x = x.view(x.shape[0], -1)
                x = self.dropout(x)
                return self.mlp(x)

Very little code is required in addition to the actual torch module.
The outer class, which provides the sensAI `VectorModel` features, serves mainly to hold the parameters, and the inner class inheriting from `VectorTorchModel` serves as a factory for the `torch.nn.Module`, providing us with the input and output dimensions (number of input columns and number of classes respectively) based on the data. Because we take the dimensions directly from the input, this model could easily process other image sizes than 28x28 and we furthermore end up with fewer magic numbers in the code.

The inner class `InputTensoriser`, which is instantiated and passed as the input tensoriser for the model, serves to convert the input data frame into a tensor. It could perform arbitrary computations in order to produce, from a data frame with N rows, one or more tensors of length N (first dimension equal to N) that will ultimately be fed to the neural network.

Let's instantiate our model and see how it performs.

In [ ]:
nnOptimiserParams = sensai.torch.NNOptimiserParams(optimiser=sensai.torch.Optimiser.ADAMW, optimiserLR=0.01, batchSize=1024, 
    earlyStoppingEpochs=3)
cnnModel = CnnModel(cuda=False, kernelSize=5, numConv=32, poolingKernelSize=2, mlpHiddenDims=(200,20),
    nnOptimiserParams=nnOptimiserParams).withName("CNN")

evalData = evalUtil.performSimpleEvaluation(cnnModel)

The model does slightly improve upon the MLP model we evaluated earlier.

In [ ]:
comparisonData = evalUtil.compareModels([torchMLPModel, cnnModel, randomForestModel], fitModels=False)
comparisonData.resultsDF

Could the CNN model have produced even better results? Let's take a look at some examples where the CNN model went wrong.

In [ ]:
misclassified = evalData.getMisclassifiedTriplesPredTrueInput()
fig, axs = plt.subplots(nrows=3, ncols=3, figsize=(9,9))
for i, (predClass, trueClass, input) in enumerate(misclassified[:9]):
    axs[i//3][i%3].imshow(reshape2DImage(input), cmap="binary")
    axs[i//3][i%3].set_title(f"{trueClass} misclassified as {predClass}")
plt.tight_layout()

While some of these examples are indeed ambiguous, there still is room for improvement.